In [ ]:
import os

print(os.getcwd())


In [33]:
import asyncio
import numpy as np
import math
from TestClient import test_client

from sidecar import Sidecar
from ipyleaflet import *
from ipywidgets import Text

from colorhash import ColorHash

# generally used geo measurement function
def measure(lat1, lon1, lat2, lon2) :  
    R = 6378.137 # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d * 1000

icon_dict = { 
    test_client.UserStatus.NON_GROUP_MEMBER : 
    Icon(icon_url='http://getdrawings.com/free-icon/man-icon-64.png', icon_size=[38, 38], icon_anchor=[19,19]),
    test_client.UserStatus.GROUP_MEMBER : 
    Icon(icon_url='http://getdrawings.com/free-icon/man-icon-58.png', icon_size=[38, 38], icon_anchor=[19,19]),
    test_client.UserStatus.GROUP_LEADER : 
    Icon(icon_url='http://getdrawings.com/free-icon/leadership-icon-png-62.png', icon_size=[38, 38], icon_anchor=[19,19]) 
}

snu_position = (37.4556699,126.9533264)
num_clients = 2


m = Map(center=snu_position, zoom=15, basemap=basemaps.Esri.WorldStreetMap)

markers = []
group_layers = {}

for i in range(num_clients):
    markers.append(Marker(
        location=m.center, 
        icon=icon_dict[test_client.UserStatus.NON_GROUP_MEMBER], 
        draggable=False))
    m += markers[i]
    
sc = Sidecar(title='Sidecar Output')
with sc:
    display(m)
    
def client_callback(clients):
    group_dict = {}
    for i in range(num_clients):
        if clients[i].position_from_server is not None:
            markers[i].location = list(clients[i].position_from_server)
        markers[i].icon = icon_dict[clients[i].status]
        if markers[i].popup is None:
            markers[i].popup = Text()
        markers[i].popup.value = 'GPS request count : ' + str(clients[i].gps_request_count)
        markers[i].title = 'GPS request count : ' + str(clients[i].gps_request_count)
        if clients[i].group_id is not None and clients[i].status is not test_client.UserStatus.NON_GROUP_MEMBER:
            group_dict.setdefault(clients[i].group_id, []).append(i)
    
    # Remove group layer if it's gone
    for group_id, layer in group_layers.items():
        if group_id not in group_dict:
            m.remove_layer(layer)
    
    # Add group layer if there's a new one
    for group_id, id_list in group_dict.items():
        sw = (90,180)
        ne = (0,0)
        
        for i in id_list:
            if clients[i].position_from_server is not None:
                sw = np.minimum(sw, clients[i].position_from_server)
                ne = np.maximum(ne, clients[i].position_from_server)
        
            
        if group_id not in group_layers:
            group_layers[group_id] = Rectangle(bounds=(list(sw), list(ne)), color=ColorHash(group_id).hex)
            m.add_layer(group_layers[group_id])
        else:
            group_layers[group_id] = Rectangle(bounds=(list(sw), list(ne)), color=ColorHash(group_id).hex)
try:
    loop = asyncio.get_running_loop()
except RuntimeError:   
    loop = asyncio.get_event_loop()
    
task = asyncio.create_task(test_client.execute(loop, test_client.PositionUpdateTest, num_clients, 'http://localhost:8080/', client_callback))

In [34]:
print("Start task")

Start task
[37.46042990477865, 126.95303972782865]
[37.46042990477865, 126.95303972782865]
[37.46042990477865, 126.95303972782865]
[37.46042990477865, 126.95303972782865]
[37.45766482932912, 126.95785485332568]
[37.45766482932912, 126.95785485332568]
[37.45766482932912, 126.95785485332568]
[37.45766482932912, 126.95785485332568]
[37.46042990477865, 126.95303972782865]
[37.46042990477865, 126.95303972782865]
[37.46042990477865, 126.95303972782865]
[37.46042990477865, 126.95303972782865]
[37.45766482932912, 126.95785485332568]
[37.45766482932912, 126.95785485332568]
[37.45766482932912, 126.95785485332568]
[37.45766482932912, 126.95785485332568]


In [32]:
# You must cancel existing task here to re-run a script above.
# Resetting a event loop is not going to work properly 
# since it's shared with a IPython kernel
task.cancel()
sc.close()


[37.453898132753764, 126.95430650798261]
[37.453898132753764, 126.95430650798261]
[37.453898132753764, 126.95430650798261]
[37.453898132753764, 126.95430650798261]
[37.45203339360303, 126.95541296172983]
[37.45203339360303, 126.95541296172983]
[37.45203339360303, 126.95541296172983]
[37.45203339360303, 126.95541296172983]


Task exception was never retrieved
future: <Task finished coro=<PositionUpdateTest.__get_position() done, defined at C:\gh\snu-amc-s19-team7\Server\TestClient\test_client.py:133> exception=ClientConnectionError('Connector is closed.')>
Traceback (most recent call last):
  File "C:\gh\snu-amc-s19-team7\Server\TestClient\test_client.py", line 134, in __get_position
    async with self.session.get(self.target_address + "user-data", params={'id' : client.id}) as resp:
  File "C:\Users\vleej\Anaconda3\lib\site-packages\aiohttp\client.py", line 1005, in __aenter__
    self._resp = await self._coro
  File "C:\Users\vleej\Anaconda3\lib\site-packages\aiohttp\client.py", line 476, in _request
    timeout=real_timeout
  File "C:\Users\vleej\Anaconda3\lib\site-packages\aiohttp\connector.py", line 525, in connect
    raise ClientConnectionError("Connector is closed.")
aiohttp.client_exceptions.ClientConnectionError: Connector is closed.
